## Run pipeline templates

These are templates for running the pipline. Note this depends on preprocessed DepMap data. See *run01-preprocess_data.ipynb* for such run.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ceres_infer.session import workflow
from ceres_infer.models import model_infer_iter_ens

/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Using TensorFlow backend.


In [3]:
import logging
logging.basicConfig(level=logging.INFO)

In [10]:
# get gene set 16
import re
with open('../out/20.0216 feat/reg_rf_boruta/network/gs.txt','r') as f:
    for lines in f:
        if lines.startswith('gene_set_16'):
            l = lines
            gs16 = re.findall(r'\w+', lines)[1:]

In [15]:
# Parameters
params = {
    # directories
    'outdir_run': '../out/20.0926 feat Sanger/reg_rf_boruta_gs16/', # output dir for the run
    'outdir_modtmp': '../out/20.0926 feat Sanger/reg_rf_boruta_gs16/model_perf/', # intermediate files for each model
    'indir_dmdata_Q3': '../out/20.0925 proc_data/gene_effect/dm_data_match_sanger.pkl', # pickled preprocessed DepMap Q3 data
    'indir_dmdata_external': '../out/20.0925 proc_data/gene_effect/dm_data_sanger.pkl', # pickled external validation data
    'indir_genesets': '../data/gene_sets/',
    'indir_landmarks': None, # csv file of landmarks [default: None]

    # notes
    'session_notes': 'regression model, with random forest (iterative) and boruta feature selection; \
    run on selective dependent genes (CERES std > 0.25 and CERES range > 0.6)',

    # data
    'external_data_name': 'Sanger', # name of external validation dataset
    'opt_scale_data': True, # scale input data True/False
    'opt_scale_data_types': '\[(?:RNA-seq|CN)\]', # data source types to scale; in regexp
    'model_data_source': ['CERES','RNA-seq','CN','Mut','Lineage'],
    'anlyz_set_topN': 10, # for analysis set how many of the top features to look at
    'perm_null': 1000, # number of samples to get build the null distribution, for corr
    'useGene_dependency': False, # whether to use CERES gene dependency (true) or gene effect (false)
    'scope': gs16, # scope for which target genes to run on; list of gene names, or 'all', 'differential'

    # model
    'model_name': 'rf',
    'model_params': {'n_estimators':1000,'max_depth':15,'min_samples_leaf':5,'max_features':'log2'},
    'model_paramsgrid': {},
    'model_pipeline': model_infer_iter_ens,
    'pipeline_params': {},
    
    # pipeline
    'parallelize': False, # parallelize workflow
    'processes': 1, # number of cpu processes to use
    
    # analysis
    'metric_eval': 'score_test',  # metric in model_results to evaluate, e.g. score_test, score_oob
    'thresholds': {'score_rd10': 0.1,  # score of reduced model - threshold for filtering
                   'recall_rd10': 0.95},  # recall of reduced model - threshold for filtering
    'min_gs_size': 4 # minimum gene set size, to be derived
}

In [16]:
# Run just the inference
wf = workflow(params)
pipeline = ['load_processed_data', 'infer']
wf.create_pipe(pipeline)
wf.run_pipe()

INFO:root:Loading preprocessed data...
INFO:root:Running model building and inference...
 72%|███████▏  | 47/65 [9:20:17<53:51, 179.55s/it]    /Users/boyangzhao/Dropbox/Industry/Quantalarity/client Penn/proj_ceres/github/cnp_dev/src/ceres_infer/data.py:353: UserWarning: gene PPIEL not found in shared CERES dataset...
  warnings.warn('gene %s not found in shared CERES dataset...' % gene)
 74%|███████▍  | 48/65 [9:20:17<35:37, 125.76s/it]/Users/boyangzhao/Dropbox/Industry/Quantalarity/client Penn/proj_ceres/github/cnp_dev/src/ceres_infer/data.py:353: UserWarning: gene SNORA55 not found in shared CERES dataset...
  warnings.warn('gene %s not found in shared CERES dataset...' % gene)
 75%|███████▌  | 49/65 [9:20:18<23:29, 88.09s/it] /Users/boyangzhao/Dropbox/Industry/Quantalarity/client Penn/proj_ceres/github/cnp_dev/src/ceres_infer/data.py:353: UserWarning: gene OXCT2P1 not found in shared CERES dataset...
  warnings.warn('gene %s not found in shared CERES dataset...' % gene)
100%|███████

In [17]:
# Run analysis, based on pre-existing inference
wf = workflow(params)
pipeline = ['load_processed_data', 'load_model_results', 
            'analyze', 'analyze_filtered', 'derive_genesets', 'run_Rscripts']
wf.create_pipe(pipeline)
wf.run_pipe()

INFO:root:Loading preprocessed data...
INFO:root:Loading model results...
INFO:root:Analyzing model results...
/Users/boyangzhao/Dropbox/Industry/Quantalarity/client Penn/proj_ceres/github/cnp_dev/src/ceres_infer/analyses.py:303: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  feat_summary = varExp_noNeg.groupby('target')['target', 'score_rd', 'score_full'].first()
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/pandas/plotting/_matplotlib/core.py:328: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = self.plt.figure(figsize=self.figsize)
/Users/boyangzhao/Dropbox/Industry/Quantalarity/client Penn/proj_ceres/github/cnp_dev/src/ceres_infer/analyses.py:35: RuntimeWarning:

/Users/boyangzhao/Dropbox/Industry/Quantalarity/client Penn/proj_ceres/github/cnp_dev/src/ceres_infer/analyses.py:397: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
/Users/boyangzhao/Dropbox/Industry/Quantalarity/client Penn/proj_ceres/github/cnp_dev/src/ceres_infer/analyses.py:406: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
/Users/boyangzhao/Dropbox/Industry/Quantalarity/client Penn/proj_ceres/github/cnp_dev/src/ceres_infer/analyses.py:414: RuntimeWarning: More than 20 figures have been opened. Figures created thro